# API Decoder

In [5]:
from sarpyx.processor.core.decode import S1L0Decoder
from pathlib import Path
import pandas as pd

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

def find_dat_file(folder: Path, pol: str) -> Path:
    """Find the .dat file in a SAFE folder."""
    assert folder.exists() and folder.is_dir(), f'Invalid folder: {folder}'
    
    for file in folder.iterdir():
        if (file.suffix == '.dat' and 
            'annot' not in file.name and 
            'index' not in file.name and 
            pol in file.name):
            return file
    
    raise FileNotFoundError(f'No valid .dat file found in {folder}')

# Setup paths
cwd = Path.cwd().parent
data_dir = cwd / 'data'
output_dir = cwd / 'processed_data'

# Find first SAFE folder and .dat file
safe_folders = [f for f in data_dir.iterdir() if f.is_dir() and f.name.endswith('.SAFE')]
if not safe_folders:
    raise FileNotFoundError('No SAFE folders found')

input_file = find_dat_file(safe_folders[0], 'vh')
print(f'📁 Input file: {input_file.name}')
print(f'📁 Output dir: {output_dir}')

📁 Input file: s1a-s1-raw-s-vh-20150107t054312-20150107t054337-004061-004e74.dat
📁 Output dir: /Data_large/marine/PythonProjects/SAR/sarpyx/processed_data


In [6]:
# Decode the file
output_dir.mkdir(exist_ok=True)

decoder = S1L0Decoder()
result = decoder.decode_file(input_file, output_dir, headers_only=False)

2025-06-05 09:37:35,620 - INFO - Processing file: /Data_large/marine/PythonProjects/SAR/sarpyx/data/S1A_S1_RAW__0SDV_20150107T054312_20150107T054337_004061_004E74_020B.SAFE/s1a-s1-raw-s-vh-20150107t054312-20150107t054337-004061-004e74.dat
2025-06-05 09:37:35,621 - INFO - File size: 747.2 MB
2025-06-05 09:37:35,621 - INFO - Starting full decode process...
2025-06-05 09:37:35,622 - INFO - Starting decode process for: /Data_large/marine/PythonProjects/SAR/sarpyx/data/S1A_S1_RAW__0SDV_20150107T054312_20150107T054337_004061_004E74_020B.SAFE/s1a-s1-raw-s-vh-20150107t054312-20150107t054337-004061-004e74.dat
decoded: 47591 packets [02:44, 289.10 packets/s]
2025-06-05 09:40:20,242 - INFO - Decoded 47591 records from file
2025-06-05 09:40:20,242 - WARNING - Incomplete sub-commutated data cycle: 0
2025-06-05 09:40:20,244 - WARNING - Incomplete sub-commutated data cycle: 28
2025-06-05 09:40:20,247 - WARNING - Incomplete sub-commutated data cycle: 58
2025-06-05 09:40:20,249 - WARNING - Incomplete s

In [ ]:
r = result['burst_data'][1]
echo = r['echo']
print(f'📊 Echo shape: {echo.shape}'
      f'\n📊 Echo dtype: {echo.dtype}'
      f'\n📊 Echo size: {echo.size}')
metadata = r['metadata']
metadata

📊 Echo shape: (40722, 25724)
📊 Echo dtype: complex64
📊 Echo size: 1047532728


,packet_version_number,packet_type,secondary_header_flag,pid,pcat,sequence_flags,packet_sequence_count,packet_data_length,coarse_time,fine_time,sync_marker,data_take_id,ecc_num,test_mode,rx_channel_id,instrument_configuration_id,data_word_index,space_packet_count,pri_count,error_flag,baq_mode,baq_block_length,range_decimation,rx_gain,tx_ramp_rate,tx_pulse_start_freq,tx_pulse_length,rank,pri,swst,swl,ssb_flag,polarization,temperature_compensation,elevation_beam_address,azimuth_beam_address,sas_test,cal_type,calibration_beam_address,cal_mode,tx_pulse_number,signal_type,swap,swath_number,number_of_quads,signal_type_name,data_take_hex,samples_per_line,polarization_name,temp_comp_name,sync_marker_valid,baq_mode_valid,packet_version_valid
0,0,0,True,65,12,3,6021,20329,1104644611,0.543404,892270675,10283296,1,0,1,3,12,6021,9916,False,12,31,1,-6.0,1.927379e+12,-4.380134e+07,0.000045,9,0.000535,0.000131,0.000258,False,7,0,0,0,0,0,0,0,0,0,True,0,12862,echo,0x009CE920,25724,NaN,disabled,True,True,True
1,0,0,True,65,12,3,6022,20217,1104644611,0.543938,892270675,10283296,1,0,1,3,13,6022,9917,False,12,31,1,-6.0,1.927379e+12,-4.380134e+07,0.000045,9,0.000535,0.000131,0.000258,False,7,0,0,0,0,0,0,0,0,0,True,0,12862,echo,0x009CE920,25724,NaN,disabled,True,True,True
2,0,0,True,65,12,3,6023,20221,1104644611,0.544472,892270675,10283296,1,0,1,3,14,6023,9918,False,12,31,1,-6.0,1.927379e+12,-4.380134e+07,0.000045,9,0.000535,0.000131,0.000258,False,7,0,0,0,0,0,0,0,0,0,True,0,12862,echo,0x009CE920,25724,NaN,disabled,True,True,True
3,0,0,True,65,12,3,6024,20213,1104644611,0.545006,892270675,10283296,1,0,1,3,15,6024,9919,False,12,31,1,-6.0,1.927379e+12,-4.380134e+07,0.000045,9,0.000535,0.000131,0.000258,False,7,0,0,0,0,0,0,0,0,0,True,0,12862,echo,0x009CE920,25724,NaN,disabled,True,True,True
4,0,0,True,65,12,3,6025,20185,1104644611,0.545540,892270675,10283296,1,0,1,3,16,6025,9920,False,12,31,1,-6.0,1.927379e+12,-4.380134e+07,0.000045,9,0.000535,0.000131,0.000258,False,7,0,0,0,0,0,0,0,0,0,True,0,12862,echo,0x009CE920,25724,NaN,disabled,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40717,0,0,True,65,12,3,14390,16645,1104644633,0.534081,892270675,10283296,1,0,1,3,61,47158,51053,False,12,31,1,-6.0,1.927379e+12,-4.380134e+07,0.000045,9,0.000535,0.000131,0.000258,False,7,0,0,0,0,0,0,0,0,0,True,0,12862,echo,0x009CE920,25724,NaN,disabled,True,True,True
40718,0,0,True,65,12,3,14391,16613,1104644633,0.534615,892270675,10283296,1,0,1,3,62,47159,51054,False,12,31,1,-6.0,1.927379e+12,-4.380134e+07,0.000045,9,0.000535,0.000131,0.000258,False,7,0,0,0,0,0,0,0,0,0,True,0,12862,echo,0x009CE920,25724,NaN,disabled,True,True,True
40719,0,0,True,65,12,3,14392,16597,1104644633,0.535149,892270675,10283296,1,0,1,3,63,47160,51055,False,12,31,1,-6.0,1.927379e+12,-4.380134e+07,0.000045,9,0.000535,0.000131,0.000258,False,7,0,0,0,0,0,0,0,0,0,True,0,12862,echo,0x009CE920,25724,NaN,disabled,True,True,True
40720,0,0,True,65,12,3,14393,16621,1104644633,0.535683,892270675,10283296,1,0,1,3,64,47161,51056,False,12,31,1,-6.0,1.927379e+12,-4.380134e+07,0.000045,9,0.000535,0.000131,0.000258,False,7,0,0,0,0,0,0,0,0,0,True,0,12862,echo,0x009CE920,25724,NaN,disabled,True,True,True


In [16]:
first_row = metadata.iloc[0]
# Create DataFrame with just the first row of metadata
first_row_df = pd.DataFrame([first_row])
print(f'📊 First row DataFrame shape: {first_row_df.shape}')
print(f'📊 First row DataFrame:\n')
first_row_df.T

📊 First row DataFrame shape: (1, 53)
📊 First row DataFrame:



,0
packet_version_number,0
packet_type,0
secondary_header_flag,True
pid,65
pcat,12
sequence_flags,3
packet_sequence_count,6021
packet_data_length,20329
coarse_time,1104644611
fine_time,0.543404
